In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [1]:
import os
import gc
import sys
import math
import json
import glob
import random
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from tqdm import tqdm

from imgaug import augmenters as iaa
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [41]:
# setting up path directory
TRAIN_IMAGE_DIR = Path('/home/ubuntu/iMaterialist_Fashion_2019_at_FGVC6/data/train/')
ROOT_DIR = Path('/home/ubuntu/iMaterialist_Fashion_2019_at_FGVC6/data/')
DATA_DIR = Path('/home/ubuntu/iMaterialist_Fashion_2019_at_FGVC6/data/')
TEST_DIR = Path('/home/ubuntu/iMaterialist_Fashion_2019_at_FGVC6/data/train_1k_images/')

In [38]:
# import train file 
import pandas as pd
train = pd.read_csv(DATA_DIR/'train.csv')
print(len(train))
train.head()

333415


,ImageId,EncodedPixels,Height,Width,ClassId
0,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6
1,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0
2,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,8521389 10 8526585 30 8531789 42 8537002 46 85...,5214,3676,28
3,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,12903854 2 12909064 7 12914275 10 12919485 15 ...,5214,3676,31
4,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,10837337 5 10842542 14 10847746 24 10852951 33...,5214,3676,32


In [4]:
# extracting image metadata fom json file 
with open(ROOT_DIR/"label_descriptions.json") as f:
    label_descriptions = json.load(f)

label_names = [x['name'] for x in label_descriptions['categories']]

In [5]:
segment_df = train
segment_df['CategoryId'] = segment_df['ClassId'].str.split('_').str[0]

print("Total segments: ", len(segment_df))
segment_df.head()

Total segments:  333415


,ImageId,EncodedPixels,Height,Width,ClassId,CategoryId
0,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6068157 7 6073371 20 6078584 34 6083797 48 608...,5214,3676,6,6
1,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,6323163 11 6328356 32 6333549 53 6338742 75 63...,5214,3676,0,0
2,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,8521389 10 8526585 30 8531789 42 8537002 46 85...,5214,3676,28,28
3,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,12903854 2 12909064 7 12914275 10 12919485 15 ...,5214,3676,31,31
4,00000663ed1ff0c4e0132b9b9ac53f6e.jpg,10837337 5 10842542 14 10847746 24 10852951 33...,5214,3676,32,32


In [6]:
# Rows with the same image are grouped together because the subsequent operations perform at an image level
image_df = segment_df.groupby('ImageId')['EncodedPixels', 'CategoryId'].agg(lambda x: list(x))
size_df = segment_df.groupby('ImageId')['Height', 'Width'].mean()
image_df = image_df.join(size_df, on='ImageId')

print("Total images: ", len(image_df))
image_df.head()

Total images:  45625


,EncodedPixels,CategoryId,Height,Width
ImageId,,,,
00000663ed1ff0c4e0132b9b9ac53f6e.jpg,[6068157 7 6073371 20 6078584 34 6083797 48 60...,"[6, 0, 28, 31, 32, 32, 31, 29, 4]",5214,3676
0000fe7c9191fba733c8a69cfaf962b7.jpg,[2201176 1 2203623 3 2206071 5 2208518 8 22109...,"[33, 1]",2448,2448
0002ec21ddb8477e98b2cbb87ea2e269.jpg,[2673735 2 2676734 8 2679734 13 2682733 19 268...,"[33, 10, 23, 23]",3000,1997
0002f5a0ebc162ecfb73e2c91e3b8f62.jpg,[435 132 1002 132 1569 132 2136 132 2703 132 3...,"[10, 33, 15]",567,400
0004467156e47b0eb6de4aa6479cbd15.jpg,[132663 8 133396 25 134130 41 134868 53 135611...,"[10, 33, 31, 31, 15]",750,500


In [19]:
train_df, valid_df = train_test_split(image_df, test_size=0.2, random_state=42)

train_dataset = FashionDataset(train_df)
train_dataset.prepare()

valid_dataset = FashionDataset(valid_df)
valid_dataset.prepare()


In [22]:
print(len(valid_df))
print(len(train_df))

9125
36500


In [42]:
# We will test on 4k images, subsetting data for that

images = os.listdir(TEST_DIR)
uploaded_images = pd.DataFrame(images, columns = ['image_name'])
image_df = image_df[image_df.index.isin(uploaded_images['image_name'])]
print(image_df)

                                                                          EncodedPixels  \
ImageId                                                                                   
0000fe7c9191fba733c8a69cfaf962b7.jpg  [2201176 1 2203623 3 2206071 5 2208518 8 22109...   
000aac3870ea7c59ca0333ffa5327323.jpg  [8971124 17 8975101 49 8979077 82 8983053 116 ...   
000b3a87508b0fa185fbd53ecbe2e4c6.jpg  [291578 3 292856 8 294133 13 295410 19 296687 ...   
000b3ec2c6eaffb491a5abb72c2e3e26.jpg  [752989 6 755294 19 757599 31 759904 44 762209...   
000c5bb8ca785bce8d00e3a6c7802a8c.jpg  [191375 4 192392 12 193413 15 194436 17 195459...   
000c9b4926cd78edd4c19cbc6beba111.jpg  [337495 5 338987 13 340478 22 341969 31 343460...   
000cd2e13d1bdd28f480304d7bb9e1ca.jpg  [1522034 13 1525024 39 1528014 64 1531004 90 1...   
000e147149bf33c584afaa953bc198ed.jpg  [89782 2 90980 5 92178 8 93377 11 94575 14 957...   
000e18920575a2e59b3a0c38e6546d29.jpg  [168019 2 168766 5 169515 6 170256 2 170265 6 ...   

In [7]:
# import matterport Mask-RCNN implementation
!git clone https://www.github.com/matterport/Mask_RCNN.git;
os.chdir('Mask_RCNN')

!rm -rf .git # to prevent an error when the kernel is committed
!rm -rf images assets # to prevent displaying images at the bottom of a kernel

sys.path.append(ROOT_DIR/'Mask_RCNN')
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.


Using TensorFlow backend.


In [43]:
# Already have trained weights, we will continue on those weights
pre_trained_weight = '/home/ubuntu/iMaterialist_Fashion_2019_at_FGVC6/data/fashion20191227T1556/mask_rcnn_fashion_0003.h5'

In [44]:
# Set configuration

NUM_CATS = 46  # classification ignoring attributes (only categories)
IMAGE_SIZE = 512 # the image size is set to 512, which is the same as the size of submission masks

class FashionConfig(Config):
    NAME = "fashion"
    NUM_CLASSES = NUM_CATS + 1 # +1 for the background class
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4 
    
    BACKBONE = 'resnet50'
    
    IMAGE_MIN_DIM = IMAGE_SIZE
    IMAGE_MAX_DIM = IMAGE_SIZE    
    IMAGE_RESIZE_MODE = "none"
    
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)

    # STEPS_PER_EPOCH should be the number of instances divided by (GPU_COUNT*IMAGES_PER_GPU), and so should VALIDATION_STEPS;
    STEPS_PER_EPOCH = 5500
    VALIDATION_STEPS = 100
    
config = FashionConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 4
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                59
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              none
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                

In [45]:
# custom function to resize an image, we are resizing image to 512X512; suitable for submission
def resize_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA)  
    return img

In [46]:
#  MaskRCNN Class

class FashionDataset(utils.Dataset):

    def __init__(self, df):
        super().__init__(self)
        
        # Add classes
        for i, name in enumerate(label_names):
            self.add_class("fashion", i+1, name)
        
        # Add images 
        for i, row in df.iterrows():
            self.add_image("fashion", 
                           image_id=row.name, 
                           path=str(TRAIN_IMAGE_DIR/row.name), 
                           labels=row['CategoryId'],
                           annotations=row['EncodedPixels'], 
                           height=row['Height'], width=row['Width'])

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path'], [label_names[int(x)] for x in info['labels']]
    
    def load_image(self, image_id):
        return resize_image(self.image_info[image_id]['path'])

    def load_mask(self, image_id):
        info = self.image_info[image_id]
                
        mask = np.zeros((IMAGE_SIZE, IMAGE_SIZE, len(info['annotations'])), dtype=np.uint8)
        labels = []
        
        for m, (annotation, label) in enumerate(zip(info['annotations'], info['labels'])):
            sub_mask = np.full(info['height']*info['width'], 0, dtype=np.uint8)
            annotation = [int(x) for x in annotation.split(' ')]
            
            for i, start_pixel in enumerate(annotation[::2]):
                sub_mask[start_pixel: start_pixel+annotation[2*i+1]] = 1

            sub_mask = sub_mask.reshape((info['height'], info['width']), order='F')
            sub_mask = cv2.resize(sub_mask, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
            
            mask[:, :, m] = sub_mask
            labels.append(int(label)+1)
            
        return mask, np.array(labels)

In [47]:
# Prediction, this cell defines InferenceConfig and loads the best trained model.

class InferenceConfig(FashionConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

model = modellib.MaskRCNN(mode='inference', config=inference_config,model_dir=ROOT_DIR)

model.load_weights(pre_trained_weight, by_name=True)

Re-starting from epoch 3


In [48]:
# refine overlapped mask
def refine_masks(masks, rois):
    areas = np.sum(masks.reshape(-1, masks.shape[-1]), axis=0)
    mask_index = np.argsort(areas)
    union_mask = np.zeros(masks.shape[:-1], dtype=bool)
    for m in mask_index:
        masks[:, :, m] = np.logical_and(masks[:, :, m], np.logical_not(union_mask))
        union_mask = np.logical_or(masks[:, :, m], union_mask)
    for m in range(masks.shape[-1]):
        mask_pos = np.where(masks[:, :, m]==True)
        if np.any(mask_pos):
            y1, x1 = np.min(mask_pos, axis=1)
            y2, x2 = np.max(mask_pos, axis=1)
            rois[m, :] = [y1, x1, y2, x2]
    return masks, rois

In [49]:
pred_file = image_df.reset_index().iloc[:,:3]
print(len(pred_file))
sample_df = pd.read_csv(DATA_DIR/"sample_submission.csv")

1000


In [50]:
def trim_masks(masks, rois, class_ids):
    class_pos = np.argsort(class_ids)
    class_rle = to_rle(np.sort(class_ids))
    
    pos = 0
    for i, _ in enumerate(class_rle[::2]):
        previous_pos = pos
        pos += class_rle[2*i+1]
        if pos-previous_pos == 1:
            continue 
        mask_indices = class_pos[previous_pos:pos]
        
        union_mask = np.zeros(masks.shape[:-1], dtype=bool)
        for m in mask_indices:
            masks[:, :, m] = np.logical_and(masks[:, :, m], np.logical_not(union_mask))
            union_mask = np.logical_or(masks[:, :, m], union_mask)
        for m in mask_indices:
            mask_pos = np.where(masks[:, :, m]==True)
            if np.any(mask_pos):
                y1, x1 = np.min(mask_pos, axis=1)
                y2, x2 = np.max(mask_pos, axis=1)
                rois[m, :] = [y1, x1, y2, x2]
            
    return masks, rois

In [51]:
# Convert data to run-length encoding
def to_rle(bits):
    rle = []
    pos = 1
    for bit, group in itertools.groupby(bits):
        group_list = list(group)
        if bit:
            rle.extend([pos, sum(group_list)])
        pos += len(group_list)
    return rle

In [52]:
%%time
sub_list = []
missing_count = 0
for i, row in tqdm(pred_file.iterrows(), total=len(pred_file)):
    image = resize_image(str(DATA_DIR/'train_1k_images'/row['ImageId']))
    print(image)
    result = model.detect([image])[0]
    if result['masks'].size > 0:
        masks, _ = trim_masks(result['masks'], result['rois'], result['class_ids'])
        for m in range(masks.shape[-1]):
            mask = masks[:, :, m].ravel(order='F')
            rle = to_rle(mask)
            label = result['class_ids'][m] - 1
            sub_list.append([row['ImageId'], ' '.join(list(map(str, rle))), label])
    else:
        # The system does not allow missing ids, this is an easy way to fill them
        sub_list.append([row['ImageId'], '1 1', 23])
        missing_count += 1




  0%|          | 0/1000 [00:00<?, ?it/s]

[[[194 196 195]
  [195 197 196]
  [195 197 196]
  ...
  [222 224 223]
  [222 224 223]
  [222 224 223]]

 [[194 196 195]
  [194 196 195]
  [195 197 196]
  ...
  [222 224 223]
  [223 225 224]
  [222 224 223]]

 [[194 196 195]
  [194 196 195]
  [195 197 196]
  ...
  [223 225 224]
  [224 226 225]
  [222 224 223]]

 ...

 [[183 183 191]
  [183 183 191]
  [182 182 190]
  ...
  [201 203 218]
  [200 202 217]
  [198 200 215]]

 [[182 182 190]
  [182 182 190]
  [182 182 190]
  ...
  [199 201 216]
  [199 201 216]
  [200 202 217]]

 [[182 182 190]
  [182 182 190]
  [182 182 190]
  ...
  [201 203 218]
  [200 202 217]
  [199 201 216]]]





  0%|          | 1/1000 [00:01<29:02,  1.74s/it]

[[[25 28 10]
  [59 80 15]
  [47 61 16]
  ...
  [27 31  9]
  [21 24  7]
  [32 36  9]]

 [[23 25  8]
  [58 77 15]
  [48 61 26]
  ...
  [17 20  2]
  [23 26  6]
  [36 43  3]]

 [[31 36 13]
  [59 70 15]
  [42 65 37]
  ...
  [26 24  6]
  [24 29  7]
  [29 33  8]]

 ...

 [[74 89  7]
  [56 79  3]
  [43 69  6]
  ...
  [74 52 14]
  [64 42 13]
  [46 23  7]]

 [[47 71 11]
  [51 77  8]
  [54 81 10]
  ...
  [80 55 18]
  [64 42  8]
  [45 32  6]]

 [[56 69  9]
  [48 68 13]
  [41 66  8]
  ...
  [89 62 16]
  [86 53 13]
  [42 34 10]]]





  0%|          | 2/1000 [00:02<24:05,  1.45s/it]

...

[[[ 33  33  33]
  [ 33  33  33]
  [ 33  33  33]
  ...
  [ 24  38  77]
  [ 27  36  77]
  [ 31  34  77]]

 [[ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]
  ...
  [ 24  38  77]
  [ 25  39  78]
  [ 24  37  76]]

 [[ 28  28  28]
  [ 28  28  28]
  [ 28  28  28]
  ...
  [ 34  39  81]
  [ 28  39  79]
  [ 22  38  76]]

 ...

 [[ 24  24  24]
  [ 23  23  23]
  [ 23  23  23]
  ...
  [ 94  71  78]
  [ 93  70  78]
  [ 92  69  77]]

 [[ 28  28  28]
  [ 26  26  26]
  [ 23  23  23]
  ...
  [128 106 112]
  [126 104 110]
  [124 102 108]]

 [[ 23  23  23]
  [ 23  23  23]
  [ 23  23  23]
  ...
  [183 161 160]
  [188 167 166]
  [181 160 159]]]





100%|█████████▉| 999/1000 [09:36<00:00,  1.62it/s]

[[[178 178 178]
  [178 178 178]
  [178 178 178]
  ...
  [184 176 175]
  [187 179 175]
  [212 205 197]]

 [[182 176 178]
  [182 176 178]
  [182 176 178]
  ...
  [184 176 175]
  [191 183 179]
  [208 201 193]]

 [[185 174 178]
  [185 174 178]
  [185 174 178]
  ...
  [186 177 177]
  [195 186 182]
  [204 197 188]]

 ...

 [[ 89  10   5]
  [ 89  10   5]
  [ 88  12   6]
  ...
  [146  36  17]
  [135  26  14]
  [130  24  15]]

 [[ 90  12   7]
  [ 90  12   7]
  [ 89  11   6]
  ...
  [124  26  17]
  [136  30  18]
  [133  22  12]]

 [[ 98  13  10]
  [ 98  13  10]
  [ 95  10   7]
  ...
  [131  32  21]
  [135  29  12]
  [138  33  14]]]





100%|██████████| 1000/1000 [09:36<00:00,  1.73it/s]

CPU times: user 10min 39s, sys: 2min 37s, total: 13min 17s
Wall time: 9min 36s


In [53]:
# save predicted data in submission file to upload in Kaggle
pred_df = pd.DataFrame(sub_list, columns=pred_file.columns.values)
print("Total image results: ", pred_df['ImageId'].nunique())
print("Missing Images: ", missing_count)

pred_df.head()

Total image results:  1000
Missing Images:  0


,ImageId,EncodedPixels,CategoryId
0,0000fe7c9191fba733c8a69cfaf962b7.jpg,95839 4 96350 6 96861 9 97373 10 97885 11 9839...,33
1,0000fe7c9191fba733c8a69cfaf962b7.jpg,61746 28 62182 176 62687 197 63198 199 63708 2...,1
2,0000fe7c9191fba733c8a69cfaf962b7.jpg,58802 9 59291 14 59307 20 59800 42 60308 49 60...,2
3,000aac3870ea7c59ca0333ffa5327323.jpg,94969 9 95470 28 95974 41 96480 50 96986 58 97...,31
4,000aac3870ea7c59ca0333ffa5327323.jpg,99204 13 99710 23 100171 78 100676 86 101184 9...,10


In [54]:
pred_df.to_csv('pred_1k_df.csv')

In [55]:
# Rows with the same image grouped together
pred_df2 = pred_df.groupby('ImageId')['EncodedPixels', 'CategoryId'].agg(lambda x: list(x))
pred_df2.head()

,EncodedPixels,CategoryId
ImageId,,
0000fe7c9191fba733c8a69cfaf962b7.jpg,[95839 4 96350 6 96861 9 97373 10 97885 11 983...,"[33, 1, 2]"
000aac3870ea7c59ca0333ffa5327323.jpg,[94969 9 95470 28 95974 41 96480 50 96986 58 9...,"[31, 10, 33, 31]"
000b3a87508b0fa185fbd53ecbe2e4c6.jpg,[111236 3 111750 3 112263 4 112775 7 113287 10...,"[33, 24, 10, 13, 31, 19, 21, 33, 21]"
000b3ec2c6eaffb491a5abb72c2e3e26.jpg,[131222 1 131734 2 132246 3 132758 5 133271 5 ...,"[31, 31, 33, 23, 10, 23, 43, 18, 6, 1, 23]"
000c5bb8ca785bce8d00e3a6c7802a8c.jpg,[158166 5 158676 9 159188 9 159699 10 160211 1...,"[23, 23, 23, 6, 23, 23, 28, 31, 15, 23, 10]"


In [56]:
image_df.head()

,EncodedPixels,CategoryId,Height,Width
ImageId,,,,
0000fe7c9191fba733c8a69cfaf962b7.jpg,[2201176 1 2203623 3 2206071 5 2208518 8 22109...,"[33, 1]",2448,2448
000aac3870ea7c59ca0333ffa5327323.jpg,[8971124 17 8975101 49 8979077 82 8983053 116 ...,"[10, 33, 3, 31, 31, 33]",4000,6000
000b3a87508b0fa185fbd53ecbe2e4c6.jpg,[291578 3 292856 8 294133 13 295410 19 296687 ...,"[10, 33, 24]",1280,852
000b3ec2c6eaffb491a5abb72c2e3e26.jpg,[752989 6 755294 19 757599 31 759904 44 762209...,"[23, 23, 6, 1, 32, 31, 31, 33]",2310,1536
000c5bb8ca785bce8d00e3a6c7802a8c.jpg,[191375 4 192392 12 193413 15 194436 17 195459...,"[23, 23, 6, 4, 31, 28]",1024,683


In [57]:
image_df.shape,pred_df2.shape

((1000, 4), (1000, 2))

In [58]:
combine_df = image_df.reset_index().merge(pred_df2.reset_index(), how='inner', left_on='ImageId', right_on='ImageId').loc[:,['ImageId','CategoryId_x','CategoryId_y']]

In [59]:
combine_df.columns = ['ImageId','Truth','Prediction']
combine_df.head()

,ImageId,Truth,Prediction
0,0000fe7c9191fba733c8a69cfaf962b7.jpg,"[33, 1]","[33, 1, 2]"
1,000aac3870ea7c59ca0333ffa5327323.jpg,"[10, 33, 3, 31, 31, 33]","[31, 10, 33, 31]"
2,000b3a87508b0fa185fbd53ecbe2e4c6.jpg,"[10, 33, 24]","[33, 24, 10, 13, 31, 19, 21, 33, 21]"
3,000b3ec2c6eaffb491a5abb72c2e3e26.jpg,"[23, 23, 6, 1, 32, 31, 31, 33]","[31, 31, 33, 23, 10, 23, 43, 18, 6, 1, 23]"
4,000c5bb8ca785bce8d00e3a6c7802a8c.jpg,"[23, 23, 6, 4, 31, 28]","[23, 23, 23, 6, 23, 23, 28, 31, 15, 23, 10]"


In [60]:
combine_df['Truth'] = combine_df['Truth'].map(lambda x:[int(y) for y in x])

In [61]:
combine_df.head()

,ImageId,Truth,Prediction
0,0000fe7c9191fba733c8a69cfaf962b7.jpg,"[33, 1]","[33, 1, 2]"
1,000aac3870ea7c59ca0333ffa5327323.jpg,"[10, 33, 3, 31, 31, 33]","[31, 10, 33, 31]"
2,000b3a87508b0fa185fbd53ecbe2e4c6.jpg,"[10, 33, 24]","[33, 24, 10, 13, 31, 19, 21, 33, 21]"
3,000b3ec2c6eaffb491a5abb72c2e3e26.jpg,"[23, 23, 6, 1, 32, 31, 31, 33]","[31, 31, 33, 23, 10, 23, 43, 18, 6, 1, 23]"
4,000c5bb8ca785bce8d00e3a6c7802a8c.jpg,"[23, 23, 6, 4, 31, 28]","[23, 23, 23, 6, 23, 23, 28, 31, 15, 23, 10]"


In [62]:
combine_df['correct']= combine_df['Truth']
combine_df['missed']= combine_df['Truth']
combine_df['extra']= combine_df['Truth']

In [63]:
for i in range(len(combine_df)):
  combine_df.loc[i,'correct'] = (list(set(combine_df.iloc[i,1]).intersection(combine_df.iloc[i,2])))
  combine_df.loc[i,'missed'] = (list(set(combine_df.iloc[i,1]) - set(combine_df.iloc[i,2])))
  combine_df.loc[i,'extra'] = (list(set(combine_df.iloc[i,2]) - set(combine_df.iloc[i,1])))

In [64]:
combine_df.head()

,ImageId,Truth,Prediction,correct,missed,extra
0,0000fe7c9191fba733c8a69cfaf962b7.jpg,"[33, 1]","[33, 1, 2]","[33, 1]",[],[2]
1,000aac3870ea7c59ca0333ffa5327323.jpg,"[10, 33, 3, 31, 31, 33]","[31, 10, 33, 31]","[33, 10, 31]",[3],[]
2,000b3a87508b0fa185fbd53ecbe2e4c6.jpg,"[10, 33, 24]","[33, 24, 10, 13, 31, 19, 21, 33, 21]","[24, 33, 10]",[],"[21, 19, 13, 31]"
3,000b3ec2c6eaffb491a5abb72c2e3e26.jpg,"[23, 23, 6, 1, 32, 31, 31, 33]","[31, 31, 33, 23, 10, 23, 43, 18, 6, 1, 23]","[33, 1, 6, 23, 31]",[32],"[18, 10, 43]"
4,000c5bb8ca785bce8d00e3a6c7802a8c.jpg,"[23, 23, 6, 4, 31, 28]","[23, 23, 23, 6, 23, 23, 28, 31, 15, 23, 10]","[28, 31, 6, 23]",[4],"[10, 15]"


In [65]:
correct_list = []
missed_list = []
extra_list = []

for i in range(len(combine_df)):
  correct_list.extend(combine_df.iloc[i,3])
  missed_list.extend(combine_df.iloc[i,4])
  extra_list.extend(combine_df.iloc[i,5])

In [66]:
label_df = pd.DataFrame(label_names).reset_index()
label_df.columns = ['Id','Labels']
label_df.head()

,Id,Labels
0,0,"shirt, blouse"
1,1,"top, t-shirt, sweatshirt"
2,2,sweater
3,3,cardigan
4,4,jacket


In [67]:
from collections import Counter
correct_df = pd.DataFrame.from_dict(Counter(correct_list), orient='index').reset_index().sort_values(by = 0, ascending = False)\
                        .merge(label_df, left_on = 'index', right_on='Id').loc[:,[0,'Labels']]
correct_df.columns = ['Count Correct','Labels']
correct_df.head(10)

,Count Correct,Labels
0,646,neckline
1,631,sleeve
2,489,shoe
3,358,dress
4,257,pants
5,219,"top, t-shirt, sweatshirt"
6,170,pocket
7,153,collar
8,109,lapel
9,95,jacket


In [68]:
missed_df = pd.DataFrame.from_dict(Counter(missed_list), orient='index').reset_index().sort_values(by = 0, ascending = False)\
                        .merge(label_df, left_on = 'index', right_on='Id').loc[:,[0,'Id','Labels']]
missed_df.columns = ['Count Missed','Label Id','Labels']
missed_df.head()

,Count Missed,Label Id,Labels
0,139,1,"top, t-shirt, sweatshirt"
1,135,32,pocket
2,101,19,belt
3,87,0,"shirt, blouse"
4,84,35,zipper


In [69]:
extra_df = pd.DataFrame.from_dict(Counter(extra_list), orient='index').reset_index().sort_values(by = 0, ascending = False)\
                        .merge(label_df, left_on = 'index', right_on='Id').loc[:,[0,'Labels']]
extra_df.columns = ['Count Extra','Labels']
extra_df.head()

,Count Extra,Labels
0,174,pants
1,151,dress
2,146,shoe
3,137,sleeve
4,110,neckline


In [70]:
error_analysis = missed_df.merge(correct_df, how = 'left', on='Labels').fillna(0)

In [71]:
error_analysis['missed %'] = (error_analysis['Count Missed']*100)/(error_analysis['Count Missed']+error_analysis['Count Correct'])
error_analysis['total labels'] = (error_analysis['Count Missed']+error_analysis['Count Correct'])

In [72]:
error_analysis.sort_values(by = 'total labels', ascending=False).head(15)

,Count Missed,Label Id,Labels,Count Correct,missed %,total labels
7,73,33,neckline,646.0,10.152990,719.0
26,23,31,sleeve,631.0,3.516820,654.0
32,14,23,shoe,489.0,2.783300,503.0
11,55,10,dress,358.0,13.317191,413.0
0,139,1,"top, t-shirt, sweatshirt",219.0,38.826816,358.0
1,135,32,pocket,170.0,44.262295,305.0
28,22,6,pants,257.0,7.885305,279.0
9,61,28,collar,153.0,28.504673,214.0
5,77,4,jacket,95.0,44.767442,172.0
2,101,19,belt,57.0,63.924051,158.0


In [73]:
error_analysis1 = error_analysis[error_analysis['total labels']>200].sort_values(by ='missed %')

In [74]:
error_analysis1.to_csv('error_analysis1.csv')